In [472]:
import pandas as pd
import pandas_profiling
import numpy as np 

In [325]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [386]:
train = pd.read_csv("../data/modulbank/train.csv",delimiter="\t")
test = pd.read_csv("../data/modulbank/test.csv",delimiter="\t")

In [387]:
test.drop(["Unnamed: 0"],axis=1,inplace=True)
train.drop(["Unnamed: 0"],axis=1,inplace=True)

In [388]:
train["0"].value_counts(normalize=True)

0    0.822197
1    0.177803
Name: 0, dtype: float64

In [389]:
all_dataframe = pd.concat([train,test])

In [332]:
report = pandas_profiling.ProfileReport(all_dataframe)

In [333]:
report.to_file("report_all.html")

In [390]:
rejected = report.get_rejected_variables()

According to the report shows that some features are constant

In [391]:
all_dataframe.drop(rejected,axis=1,inplace=True)

In [392]:
train,test = all_dataframe[:len(train)],all_dataframe[len(train):]

In [393]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV

In [382]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8,random_state=42,shuffle=True)


In [396]:
X_train_new,Y_train_new = train.drop(["0"],axis=1),train["0"]
myIterator=skf.split(X_train_new,Y_train_new)

In [397]:
log_reg  = LogisticRegression()
params = {"C":[0.01,0.1,1,5,10]}
grid_search = GridSearchCV(log_reg,param_grid=params,scoring="roc_auc",cv=myIterator,verbose=3,n_jobs=1)

In [398]:
grid_search.fit(X_train_new,Y_train_new)

Fitting 8 folds for each of 5 candidates, totalling 40 fits
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7119386694142167, total=   1.2s
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s


[CV] ................. C=0.01, score=0.7212224715717962, total=   1.2s
[CV] C=0.01 ..........................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.6s remaining:    0.0s


[CV] ................. C=0.01, score=0.7089897578486306, total=   1.2s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7196111087587566, total=   1.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7300517988454643, total=   1.1s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7184194475662306, total=   1.1s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7248283645166818, total=   1.0s
[CV] C=0.01 ..........................................................
[CV] ................. C=0.01, score=0.7230079586448341, total=   1.0s
[CV] C=0.1 ...........................................................
[CV] .................... C=0.1, score=0.72474206433219, total=   1.7s
[CV] C=0.1 ...........................................................
[CV] .

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  1.8min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x1142a4e60>,
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.01, 0.1, 1, 5, 10]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='roc_auc', verbose=3)

In [366]:
log_reg = LogisticRegression(**grid_search.best_params_)

In [368]:
myIterator=skf.split(X_train_new,Y_train_new)
scores1 = cross_val_score(log_reg,X_train_new,Y_train_new,cv=myIterator,scoring="roc_auc",n_jobs=1,verbose=3)


[CV]  ................................................................
[CV] ......................... , score=0.72474206433219, total=   2.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] ....................... , score=0.7321016405320085, total=   1.7s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.8s remaining:    0.0s


[CV] ....................... , score=0.7180881944738489, total=   2.0s
[CV]  ................................................................
[CV] ....................... , score=0.7286582640564941, total=   2.0s
[CV]  ................................................................
[CV] ....................... , score=0.7380662705301737, total=   1.5s
[CV]  ................................................................
[CV] ....................... , score=0.7292409071314144, total=   2.0s
[CV]  ................................................................
[CV] ....................... , score=0.7324387769124626, total=   2.2s
[CV]  ................................................................
[CV] ....................... , score=0.7221350810993733, total=   1.7s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   15.4s finished


In [369]:
print("Log regression val {:.4f}".format(scores1.mean()))

Log regression val 0.7282


In [401]:

cls = RandomForestClassifier()

In [402]:
param_grid = {"max_depth": list(range(5,40,8)),
              "max_features": [0.3,0.7,0.9],
              "min_samples_split": [ 10,20,30],
              "min_samples_leaf": [10,20,30],
              "criterion": ["gini", "entropy"]}

# run grid search
grid_search = RandomizedSearchCV(cls, param_distributions=param_grid,n_iter=20,cv=3,verbose=3,scoring="roc_auc",n_jobs=8)

In [403]:
grid_search.fit(X_train_new,Y_train_new)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.3 
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.3 
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.3 
[CV] max_depth=13, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV] max_depth=13, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV] max_depth=13, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV] max_depth=29, min_samples_leaf=20, criterion=gini, min_samples_split=20, max_features=0.9 
[CV] max_depth=29, min_samples_leaf=20, criterion=gini, min_samples_split=20, max_features=0.9 
[CV]  max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.3, score=0.7100055832460799, total=  15.8s
[CV] max_dept

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.7min


[CV]  max_depth=29, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9, score=0.6975466338015197, total=  54.0s
[CV] max_depth=29, min_samples_leaf=10, criterion=gini, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=30, max_features=0.7, score=0.7047703140698331, total=  17.0s
[CV] max_depth=29, min_samples_leaf=10, criterion=gini, min_samples_split=10, max_features=0.9 
[CV]  max_depth=29, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9, score=0.6991734301737478, total=  53.7s
[CV] max_depth=29, min_samples_leaf=10, criterion=gini, min_samples_split=10, max_features=0.9 
[CV]  max_depth=29, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9, score=0.7030199419416515, total=  54.7s
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=30, max_features=0.7, score=0.7012266599404806, total=  16.7s
[CV] max_depth

[CV]  max_depth=37, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.7053866017445031, total=  37.3s
[CV] max_depth=21, min_samples_leaf=10, criterion=gini, min_samples_split=20, max_features=0.7 
[CV]  max_depth=37, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.7152528816668016, total=  35.0s
[CV]  max_depth=37, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.7166220408398769, total=  37.4s
[CV]  max_depth=21, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.9, score=0.7055920199414123, total=  37.2s
[CV]  max_depth=21, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.9, score=0.7065368311617916, total=  37.0s
[CV]  max_depth=21, min_samples_leaf=10, criterion=gini, min_samples_split=20, max_features=0.7, score=0.7010828109599425, total=  23.9s
[CV]  max_depth=21, min_samples_leaf=10, criterion=gini, min_sample

[Parallel(n_jobs=8)]: Done  60 out of  60 | elapsed:  4.0min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=20, n_jobs=8,
          param_distributions={'max_depth': [5, 13, 21, 29, 37], 'min_samples_split': [10, 20, 30], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [10, 20, 30], 'max_features': [0.3, 0.7, 0.9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [406]:
grid_search.best_score_

0.7167775225371534

In [407]:
best_rf = RandomForestClassifier(**grid_search.best_params_)

In [409]:
myIterator=skf.split(X_train_new,Y_train_new)
scores2 = cross_val_score(best_rf,X_train_new,Y_train_new ,cv=myIterator,scoring="roc_auc",verbose=3,n_jobs=8)

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ........................ , score=0.722588719048896, total=  21.2s
[CV] ....................... , score=0.7056816417552328, total=  21.2s


[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:   22.6s remaining:  1.1min


[CV] ....................... , score=0.7047112955357017, total=  21.0s
[CV] ....................... , score=0.7056654105093788, total=  21.3s
[CV] ....................... , score=0.7318409996565562, total=  20.6s


[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   23.3s remaining:   14.0s


[CV] ....................... , score=0.7171162979821655, total=  19.8s
[CV] ....................... , score=0.7237718203432522, total=  20.5s
[CV] ....................... , score=0.7265171431830244, total=  19.5s


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   24.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   24.0s finished


In [410]:
print("RF  val {:.4f}".format(scores2.mean()))

RF  val 0.7172


In [411]:
best_rf.fit(X_train_new,Y_train_new)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=29, max_features=0.3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=30, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [412]:
X_test = test.drop(["0"],axis=1)

In [416]:
def submit(vals):
    test["id"] = test.index
    submission = pd.DataFrame.from_dict({'_ID_': test['id']})
    submission["_VAL_"]=vals[:,1]
    submission.to_csv("submission_b.csv",index=False)
    return  pd.read_csv("submission_b.csv")

In [417]:
from sklearn.ensemble import BaggingClassifier

In [428]:
clas = BaggingClassifier(log_reg,n_estimators=50,oob_score=True,bootstrap_features=True,n_jobs=8,verbose=3)
clas.fit(X_train_new,Y_train_new)

Building estimator 1 of 7 for this parallel run (total 50)...
Building estimator 1 of 7 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 1 of 6 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building estimator 2 of 7 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building estimator 2 of 7 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building estimator 2 of 6 for this parallel run (total 50)...
Building

[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:  1.3min remaining:  4.0min


Building estimator 7 of 7 for this parallel run (total 50)...


[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:  1.4min remaining:   49.1s


Building estimator 7 of 7 for this parallel run (total 50)...


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.4min finished


BaggingClassifier(base_estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=True, max_features=1.0,
         max_samples=1.0, n_estimators=50, n_jobs=8, oob_score=True,
         random_state=None, verbose=3, warm_start=False)

In [424]:
vals= np.c_[clas.oob_decision_function_,Y_train_new]

In [425]:
vals = vals[~np.isnan(vals[:,1])]

In [426]:
preds,true = vals[:,1],vals[:,2]

In [429]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_train_new,clas.oob_decision_function_[:,1])

0.7250284208539598

Ensembles of models

In [430]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier,ExtraTreesClassifier

In [434]:
extree = ExtraTreesClassifier()
rand_search = RandomizedSearchCV(extree,param_distributions=param_grid,scoring="roc_auc",cv=myIterator,n_jobs=8,verbose=3)

In [435]:
myIterator=skf.split(X_train_new,Y_train_new)
rand_search.fit(X_train_new,Y_train_new)

Fitting 8 folds for each of 10 candidates, totalling 80 fits
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV] max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3 
[CV]  max_depth=37, min_samples_leaf=10, criterion=entropy, min_samples_split=20, max_features=0.3, score=0.7173293249213136, total=  19.7s
[CV] ma

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  1.6min


[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.3, score=0.6891269471614139, total=   7.2s
[CV] max_depth=5, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.3, score=0.7073772188583553, total=   7.1s
[CV] max_depth=5, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.3, score=0.6937020413732105, total=   6.9s
[CV] max_depth=5, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=10, max_features=0.3, score=0.702977139819245, total=   6.8s
[CV] max_depth=5, min_samples_leaf=10, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=ent

[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.7015050363909235, total=  17.3s
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.7011112522523795, total=  17.2s
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.6849801774429466, total=  16.6s
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterion=entropy, min_samples_split=20, max_features=0.9, score=0.6999471021981767, total=  16.3s
[CV] max_depth=29, min_samples_leaf=20, criterion=entropy, min_samples_split=10, max_features=0.9 
[CV]  max_depth=5, min_samples_leaf=30, criterio

[Parallel(n_jobs=8)]: Done  80 out of  80 | elapsed:  5.1min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x129980150>,
          error_score='raise',
          estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=8,
          param_distributions={'max_depth': [5, 13, 21, 29, 37], 'min_samples_split': [10, 20, 30], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [10, 20, 30], 'max_features': [0.3, 0.7, 0.9]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [436]:
rand_search.best_score_

0.7249621736009902

In [437]:
extr_tree = ExtraTreesClassifier(**rand_search.best_params_)

In [438]:
preds = []
for clf in (best_rf,log_reg,extr_tree):
    print("Classifier is being trained {}".format(clf))
    clf.fit(X_train_new,Y_train_new)
    preds.append(clf.predict_proba(X_test)[:,1])
    

Classifier is being trained RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=29, max_features=0.3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=30, min_samples_split=20,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Classifier is being trained LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Classifier is being trained ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='entropy',
           max_depth=37, max_features=0.3, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=

In [439]:
np.corrcoef(preds[0],preds[1])

array([[1.        , 0.80300428],
       [0.80300428, 1.        ]])

In [440]:
np.corrcoef(preds[1],preds[2])

array([[1.        , 0.78441624],
       [0.78441624, 1.        ]])

In [441]:
np.corrcoef(preds[0],preds[2])

array([[1.       , 0.8438106],
       [0.8438106, 1.       ]])

Seems like our predictions are not completely correlated which makes it a good choice for the ensemble

In [442]:
voting_clf = VotingClassifier(estimators=[("lr",log_reg),("rf",best_rf),("extrees",extr_tree)],voting="soft")
scores3 = cross_val_score(voting_clf,X_train_new,Y_train_new,cv=myIterator,n_jobs=1,scoring="roc_auc",verbose=3)

[CV]  ................................................................
[CV] ....................... , score=0.7369305537912897, total=   9.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.3s remaining:    0.0s


[CV] ....................... , score=0.7377463503220373, total=  11.6s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   20.9s remaining:    0.0s


[CV] ........................ , score=0.729756813594727, total=  11.4s
[CV]  ................................................................
[CV] ........................ , score=0.735573245261182, total=  10.6s
[CV]  ................................................................
[CV] ....................... , score=0.7536910794013729, total=   9.6s
[CV]  ................................................................
[CV] ....................... , score=0.7334586126469044, total=  10.3s
[CV]  ................................................................
[CV] ....................... , score=0.7384208406044279, total=   9.7s
[CV]  ................................................................
[CV] ......................... , score=0.73752261139322, total=   9.6s


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  1.4min finished


In [444]:
print("Ensembles val {:.4f} ".format(scores3.mean()))

Ensembles val 0.7379 


In [445]:
voting_clf.fit(X_train_new,Y_train_new)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('rf', RandomF...timators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=None)

## Stacking

In [447]:
rf_predictions = []
et_predictions = []
log_predictions=[]
for train, val in skf.split(X_train_new,Y_train_new):
    best_rf.fit(X_train_new.iloc[train], Y_train_new[train])
    extr_tree.fit(X_train_new.iloc[train], Y_train_new[train])
    log_reg.fit(X_train_new.iloc[train],Y_train_new[train])
    rf_predictions.append([Y_train_new[val], best_rf.predict_proba(X_train_new.iloc[val])[:,1]])
    et_predictions.append([Y_train_new[val], extr_tree.predict_proba(X_train_new.iloc[val])[:,1]])
    log_predictions.append([Y_train_new[val], log_reg.predict_proba(X_train_new.iloc[val])[:,1]])
    

In [448]:
X_train_aug = X_train_new.copy()

In [450]:
X_train_aug["rf_preds"] = 0
X_train_aug["et_preds"] = 0
X_train_aug["log_preds"]=0

In [451]:
for i, fold in enumerate(skf.split(X_train_new,Y_train_new)):
    train, val = fold[0], fold[1]
    X_train_aug.iloc[val, -1] = rf_predictions[i][1]
    X_train_aug.iloc[val, -2] = et_predictions[i][1]
    X_train_aug.iloc[val, -3] = log_predictions[i][1]

In [456]:
import xgboost 

In [457]:
xgbClas = xgboost.XGBClassifier()

In [458]:
myIterator=skf.split(X_train_aug,Y_train_new)
scores_xgb = cross_val_score(xgbClas,X_train_aug,Y_train_new,cv=myIterator,n_jobs=8,scoring="roc_auc",verbose=3)

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=0.7472564489797839, total= 1.3min
[CV] ....................... , score=0.7397832540589877, total= 1.3min


[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:  1.4min remaining:  4.1min


[CV] ......................... , score=0.73902955968629, total= 1.3min
[CV] ....................... , score=0.7300376847186348, total= 1.3min
[CV] ........................ , score=0.749017892008111, total= 1.3min


[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:  1.4min remaining:   49.3s


[CV] ....................... , score=0.7387542755919001, total= 1.3min
[CV] ....................... , score=0.7386147364616453, total= 1.3min
[CV] ....................... , score=0.7333677237031501, total= 1.3min


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.4min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.4min finished


In [459]:
print("Using stacking {:.5f}".format(scores_xgb.mean()))

Using stacking 0.73948


In [468]:
xgbClas.fit(X_train_aug,Y_train_new)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [466]:
def predict_test_proba():
    X_test["rf_preds"] = 0
    X_test["et_preds"] = 0
    X_test["log_preds"]=0
    for index,clas in enumerate([best_rf,extr_tree,log_reg]):
        clas.fit(X_train_new,Y_train_new)
        res = clas.predict_proba(X_test.iloc[:,:-3])[:,1]
        X_test.iloc[:,-(index+1)] = res

In [467]:
predict_test_proba()

In [470]:
vals = xgbClas.predict_proba(X_test)

In [474]:
submit(vals)

/Users/mac/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,_ID_,_VAL_
0,0,0.153728
1,1,0.412388
2,2,0.275629
3,3,0.324806
4,4,0.653551
5,5,0.263956
6,6,0.076646
7,7,0.138943
8,8,0.096029
9,9,0.274639


## Boosting tuning

In [187]:
!pip install bayesian-optimization

  Running setup.py bdist_wheel for bayesian-optimization ... done
  Stored in directory: /Users/mac/Library/Caches/pip/wheels/c5/92/8b/a2c219cb16b9a6271dd0e72b5c9e930c32dee10908870e5512
Successfully built bayesian-optimization


Inspired by  __[this course material ](https://www.coursera.org/learn/bayesian-methods-in-machine-learning/lecture/iRLaF/bayesian-optimization)__ I decided to use bayesian optimization for my paramaters tuning

In [499]:
def xgb_evaluate(min_child_weight, colsample_bytree, max_depth,
                 subsample,
                 gamma,
                 alpha,
                 n_estimators,
                 learning_rate):
    
 
    '''According to the idea, this is our surrogate function which we will optimize '''

    params['min_child_weight'] = int(min_child_weight)
    params['cosample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    params['alpha'] = max(alpha, 0)
    params["n_estimators"] = int(n_estimators)
    params["learning_rate"] = max(learning_rate,0)


    cv_result = xgb.cv(params, xgtrain, num_boost_round=num_rounds, nfold=5,
             seed=random_state,
             callbacks=[xgb.callback.early_stop(50)])

    return cv_result['test-auc-mean'].values[-1]

In [500]:
from bayes_opt import BayesianOptimization

In [501]:
xgtrain = xgboost.DMatrix(X_train_aug,Y_train_new)

In [502]:
num_rounds = 3000
random_state = 2016
num_iter = 25
init_points = 10
params = {
    'eta': 0.1,
    'silent': 1,
    'eval_metric': 'auc',
    'verbose_eval': True,
    'seed': random_state
}

xgbBO = BayesianOptimization(xgb_evaluate, {'min_child_weight': (1, 20),
                                            'colsample_bytree': (0.1, 1),
                                            'max_depth': (3, 15),
                                            'subsample': (0.5, 1),
                                            'gamma': (0, 10),
                                            'alpha': (0, 10),
                                            "n_estimators":(50,1000),
                                            "learning_rate":(0.01,0.1),
                                            })

In [ ]:
xgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
--------------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |     gamma |   learning_rate |   max_depth |   min_child_weight |   n_estimators |   subsample | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[268]	train-auc:0.751394+0.00255534	test-auc:0.740383+0.00800751

    1 | 15m18s |    0.74038 |    6.9163 |             0.1734 |    2.0084 |          0.0626 |      6.8430 |             4.4632 |       757.6568 |      0.6534 | 
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.


Trying here RandomSearch

In [221]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

In [223]:
myIterator=skf.split(X_train_new,Y_train_new)
rand_search = RandomizedSearchCV(xgbClas,param_distributions=param_test1,scoring="roc_auc",cv=myIterator,n_jobs=8,verbose=3)

In [224]:
rand_search.fit(X_train_new,Y_train_new)

Fitting 8 folds for each of 10 candidates, totalling 80 fits
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV] max_depth=5, min_child_weight=1 .................................
[CV]  max_depth=5, min_child_weight=1, score=0.7357362634260631, total= 2.3min
[CV] max_depth=7, min_child_weight=1 .................................
[CV]  max_depth=5, min_child_weight=1, score=0.7393967622193053, total= 2.3min
[CV] max_depth=7, min_child_weight=1 .................................
[CV]  max_depth=5, min_child_weight=1, score=0.730051093139123, total= 

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  5.4min


[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV] max_depth=3, min_child_weight=1 .................................
[CV]  max_depth=3, min_child_weight=1, score=0.737274232779589, total= 1.3min
[CV] max_depth=7, min_child_weight=3 .................................
[CV]  max_depth=3, min_child_weight=1, score=0.739025090212794, total= 1.3min
[CV]  max_depth=3, min_child_weight=1, score=0.7326073967434005, total= 1.3min
[CV] max_depth=7, min_child_weight=3 .................................
[CV] max_depth=7, min_child_weight=3 .................................
[CV]  max_depth=3, min_child_weight=1, score=0.7397893701806137, total= 1.3min
[CV] max_depth=7, min_child_weight=3 .................................
[CV]  max_depth=3, min_child_weight=1, score=0.7333837839058909, total= 1.3min
[CV]  max_depth=3, min_child_weight=1, score=0.7506372528263539, total= 1.3min
[CV] max_depth=7, min_child_wei

[CV] max_depth=3, min_child_weight=5 .................................
[CV]  max_depth=7, min_child_weight=5, score=0.7334124917876633, total= 2.5min
[CV] max_depth=3, min_child_weight=5 .................................
[CV]  max_depth=7, min_child_weight=5, score=0.726888775982482, total= 2.5min
[CV] max_depth=3, min_child_weight=5 .................................
[CV]  max_depth=3, min_child_weight=5, score=0.7388973573649866, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7394407512479241, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.731445804105329, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7406096361848574, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7331849465280406, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7507050006351357, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7406344065374906, total= 1.1min
[CV]  max_depth=3, min_child_weight=5, score=0.7357162921745491, total= 1.1min

[Parallel(n_jobs=8)]: Done  80 out of  80 | elapsed: 27.4min finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x116281f10>,
          error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=10, n_jobs=8,
          param_distributions={'max_depth': range(3, 10, 2), 'min_child_weight': range(1, 6, 2)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='roc_auc', verbose=3)

In [228]:
params_1=rand_search.best_params_

In [230]:
param_test2 = {
    'max_depth':[2,3],
    'min_child_weight':range(6,10,2)
}

In [231]:
myIterator=skf.split(X_train_new,Y_train_new)
rand_search = GridSearchCV(xgbClas,param_grid=param_test2,scoring="roc_auc",cv=myIterator,n_jobs=8,verbose=3)

In [233]:
rand_search.fit(X_train_new,Y_train_new)

Fitting 8 folds for each of 4 candidates, totalling 32 fits
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV] max_depth=2, min_child_weight=6 .................................
[CV]  max_depth=2, min_child_weight=6, score=0.7363558735938801, total= 1.1min
[CV]  max_depth=2, min_child_weight=6, score=0.7423739020385504, total= 1.1min
[CV] max_depth=2, min_child_weight=8 .................................
[CV]  max_depth=2, min_child_weight=6, score=0.7303425498581531, total= 1.1min
[CV]  max_depth=2, min_child_weight=6, score=0.7394595700836968,

[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:  2.4min


[CV] max_depth=3, min_child_weight=6 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7378117457763476, total= 1.4min
[CV] max_depth=3, min_child_weight=8 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7325843436695788, total= 1.4min
[CV] max_depth=3, min_child_weight=8 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7402607820167206, total= 1.4min
[CV] max_depth=3, min_child_weight=8 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7392742045513354, total= 1.4min
[CV] max_depth=3, min_child_weight=8 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7391135947350685, total= 1.4min
[CV] max_depth=3, min_child_weight=8 .................................
[CV]  max_depth=3, min_child_weight=6, score=0.7511999360159584, total= 1.4min
[CV]  max_depth=3, min_child_weight=6, score=0.7333623706498147, total= 1.5min
[CV] max_depth=3, min

[Parallel(n_jobs=8)]: Done  28 out of  32 | elapsed:  5.1min remaining:   44.0s


[CV]  max_depth=3, min_child_weight=8, score=0.7327828008727197, total= 1.3min
[CV]  max_depth=3, min_child_weight=8, score=0.7354042808705021, total= 1.3min
[CV]  max_depth=3, min_child_weight=8, score=0.7407963001658469, total= 1.3min


[Parallel(n_jobs=8)]: Done  32 out of  32 | elapsed:  5.2min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x1174221a8>,
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'max_depth': [2, 3], 'min_child_weight': range(6, 10, 2)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [237]:
rand_search.best_params_

{'max_depth': 3, 'min_child_weight': 6}

In [241]:
clas_new = xgboost.XGBClassifier(**rand_search.best_params_)

In [245]:
param_test3 = {
    "learning_rate":[0.05,0.08,0.01,0.15]
}

In [247]:
myIterator=skf.split(X_train_new,Y_train_new)
rand_search = GridSearchCV(clas_new,param_grid=param_test3,scoring="roc_auc",cv=3,n_jobs=-1,verbose=3)

In [248]:
rand_search.fit(X_train_new,Y_train_new)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] learning_rate=0.05 ..............................................
[CV] learning_rate=0.05 ..............................................
[CV] learning_rate=0.05 ..............................................
[CV] learning_rate=0.08 ..............................................
[CV] ..... learning_rate=0.05, score=0.7353774930735459, total=  32.0s
[CV] learning_rate=0.08 ..............................................
[CV] ..... learning_rate=0.05, score=0.7425282232493831, total=  32.0s
[CV] learning_rate=0.08 ..............................................
[CV] ..... learning_rate=0.05, score=0.7370605783770933, total=  32.6s
[CV] learning_rate=0.01 ..............................................
[CV] ..... learning_rate=0.08, score=0.7344899832409982, total=  32.7s
[CV] learning_rate=0.01 ..............................................
[CV] ..... learning_rate=0.08, score=0.7426896067192693, total=  35.2s
[CV] learning_rat

[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  1.8min remaining:   21.5s


[CV] ..... learning_rate=0.15, score=0.7357492989108764, total=  35.7s
[CV] ..... learning_rate=0.15, score=0.7406190812827466, total=  36.4s


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  1.8min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.05, 0.08, 0.01, 0.15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [250]:
rand_search.best_params_

{'learning_rate': 0.05}

In [251]:
param_test4 = {
    "n_estimators":list(range(50,400,80))
}

myIterator=skf.split(X_train_new,Y_train_new)
rand_search = GridSearchCV(xgboost.XGBClassifier(learning_rate=0.05,max_depth=3,min_child_weight=6),param_grid=param_test4,scoring="roc_auc",cv=3,n_jobs=8,verbose=3)

In [252]:
rand_search.fit(X_train_new,Y_train_new)

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=50 .................................................
[CV] n_estimators=130 ................................................
[CV] n_estimators=130 ................................................
[CV] n_estimators=130 ................................................
[CV] n_estimators=210 ................................................
[CV] n_estimators=210 ................................................
[CV] ........ n_estimators=50, score=0.7338011093972159, total=  34.3s
[CV] n_estimators=210 ................................................
[CV] ........ n_estimators=50, score=0.7407672642597084, total=  34.5s
[CV] n_estimators=290 ................................................
[CV] ........ n_estimators=50, score=0.7347210105878874, total=  35.1s
[CV] n_estimators

[Parallel(n_jobs=8)]: Done   6 out of  15 | elapsed:  1.6min remaining:  2.4min


[CV] n_estimators=370 ................................................
[CV] ....... n_estimators=210, score=0.7339608055860517, total= 2.3min
[CV] n_estimators=370 ................................................
[CV] ....... n_estimators=210, score=0.7420267699379716, total= 2.4min
[CV] ....... n_estimators=210, score=0.7375583700266268, total= 2.3min
[CV] ....... n_estimators=290, score=0.7338337301641915, total= 2.9min
[CV] ....... n_estimators=290, score=0.7427756580528433, total= 2.9min
[CV] ....... n_estimators=290, score=0.7366921503242068, total= 2.5min


[Parallel(n_jobs=8)]: Done  12 out of  15 | elapsed:  4.1min remaining:  1.0min


[CV] ........ n_estimators=370, score=0.733242222362204, total= 2.8min
[CV] ....... n_estimators=370, score=0.7428755715581848, total= 2.8min
[CV] ....... n_estimators=370, score=0.7364657309278236, total= 2.3min


[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:  4.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=6, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'n_estimators': [50, 130, 210, 290, 370]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [256]:
rand_search.best_params_

{'n_estimators': 130}

In [257]:
clf = xgboost.XGBClassifier(learning_rate=0.05,min_child_weight=6,n_estimators=130)

In [259]:
myIterator=skf.split(X_train_new,Y_train_new)
score_final = cross_val_score(clf,X_train_new,Y_train_new,cv=myIterator,scoring="roc_auc",n_jobs=8,verbose=3)

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ....................... , score=0.7376096785272379, total= 1.8min
[CV] ....................... , score=0.7321531570949363, total= 1.8min


[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:  1.8min remaining:  5.5min


[CV] ....................... , score=0.7419029606733379, total= 1.8min
[CV] ....................... , score=0.7401633945415966, total= 1.8min
[CV] ....................... , score=0.7523031902631344, total= 1.8min


[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:  1.8min remaining:  1.1min


[CV] ......................... , score=0.73448974328565, total= 1.8min
[CV] ....................... , score=0.7400223639105218, total= 1.8min
[CV] ........................ , score=0.736233326012222, total= 1.8min


[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.9min remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:  1.9min finished


In [294]:
score_final.mean()

0.7393597267885796

In [262]:
clf.fit(X_train_new,Y_train_new)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=6, missing=None, n_estimators=130,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)